## 要件
- 一行目にソートボタン
- 二行目にフィルターブロック
  - 選択式のドロップダウン
  - 値範囲指定の入力ボックス
- Pythonからデータを埋め込み
- 出力HTMLは独立（他を参照せず、css, jsが埋め込まれている）

<pre>
project/  
├─ data_to_html.py      ← Pythonスクリプト（Jinja2でHTML生成）  
├─ templates/  
│   ├─ base_251020.html ← Jinja2テンプレート
│   └─ table.html
├─ static/  
│   ├─ style/  
│   │   ├─ table.css  
│   │   ├─ filter.css  
│   │   └─ sort.css  
│   └─ js/  
│       └─ table.js  
└─ output/
     └─ table.html          ← Pythonで生成されるブラウザ用HTML
</pre>

In [9]:
from jinja2 import Environment, FileSystemLoader
from pathlib import Path

# --- ディレクトリ設定 ---
try:
    BASE_DIR = Path(__file__).resolve().parent
except NameError:
    BASE_DIR = Path.cwd()

TEMPLATE_DIR = BASE_DIR / "templates"
STATIC_DIR = BASE_DIR / "static"
OUTPUT_DIR = BASE_DIR / "output"

OUTPUT_DIR.mkdir(exist_ok=True)

# --- CSSとJSをまとめて埋め込み ---
def load_static_files():
    css_dir = STATIC_DIR / "style"
    js_dir = STATIC_DIR / "js"

    # 複数CSSを結合
    embedded_css = ""
    for css_file in sorted(css_dir.glob("*.css")):
        embedded_css += f"\n/* === {css_file.name} === */\n"
        embedded_css += css_file.read_text(encoding="utf-8")

    # 複数JSを結合
    embedded_js = ""
    for js_file in sorted(js_dir.glob("*.js")):
        embedded_js += f"\n// === {js_file.name} ===\n"
        embedded_js += js_file.read_text(encoding="utf-8")

    return embedded_css, embedded_js

# --- データ定義 ---
data = [
    {"name": "tanaka", "age": 28, "city": "tokyo"},
    {"name": "takahashi", "age": 40, "city": "osaka"},
    {"name": "satou", "age": 34, "city": "osaka"},
    {"name": "suzuki", "age": 22, "city": "nagoya"},
    {"name": "takahashi", "age": 29, "city": "tokyo"},
    {"name": "takaashigani", "age": 100, "city": "suruga"},
]

columns = [
    {"key": "name", "label": "名前", "filter": "select",
     "options": sorted(set(r["name"] for r in data))},
    {"key": "age", "label": "年齢", "filter": "range"},
    {"key": "city", "label": "都市", "filter": "select",
     "options": sorted(set(r["city"] for r in data))},
]

# --- Jinja2環境 ---
env = Environment(loader=FileSystemLoader(TEMPLATE_DIR, encoding="utf-8"))
template = env.get_template("table.html")

# --- CSS/JSを読み込み ---
embedded_css, embedded_js = load_static_files()

# --- HTMLレンダリング ---
html = template.render(
    title="ユーザ一覧",
    columns=columns,
    data=data,
    embedded_css=embedded_css,
    embedded_js=embedded_js,
)

# --- 出力 ---
output_path = OUTPUT_DIR / "table.html"
output_path.write_text(html, encoding="utf-8")
print(f"✅ 出力完了: {output_path}")

✅ 出力完了: /Users/cattleya/Documents/GitHub/py_Labo/src/output/table.html
